# get_rnx

**Author:** Marilyn Braojos Gutierrez\
**Purpose:** This program aims to download and uncompress RINEX format files with GPS satellite broadcast information. The program only obtains daily files.\
**PhD Milestone:** #1: *Leverage deep learning models to GPS satellite clock bias corrections.*\
**Project:** This program is Step (1) in this PhD milestone. Obtaining the data is the first critical step.\
**References:**\
[1] https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/broadcast_ephemeris_data.html#GPShourly

# Import Libraries

In [ ]:
from ftplib import FTP_TLS                                    # https://docs.python.org/3/library/ftplib.html
import gzip                                                   # https://docs.python.org/3/library/gzip.html
import matplotlib.pyplot as plt                               # https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
import numpy as np                                            # https://numpy.org/
import os                                                     # https://docs.python.org/3/library/os.html
import time                                                   # https://docs.python.org/3/library/time.html
from unlzw import unlzw                                       # https://python-unlzw.readthedocs.io/_/downloads/en/latest/pdf/

# Connecting to NASA's CDDIS FTP Server

In [ ]:
# https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/GNSS_data_and_product_archive.html

In [ ]:
host = "gdc.cddis.eosdis.nasa.gov"                                    # FTP server host
port = 21                                                             # set port (21 is std port for FTP in passive mode)
username = "anonymous"
password = ""
timeout = 30                                                          # connection timeout (seconds)

In [ ]:
ftp = FTP_TLS(timeout=timeout)
ftp.connect(host, port)
ftp.login(username, password)
ftp.prot_p()
ftp.retrlines('LIST')                                                 # list files in server

# Fetching and Uncompressing Daily GPS Data from FTP Server

In [ ]:
def fetch_data(yr):

    '''
    Download and uncompress daily GPS broadcast RINEX files.

    Parameters:
    yr (string): year of interest YYYY.
    
    Returns:
    N/A.

    Raises: 
    Error: If connection is to server is lost. 
    '''
    
    retries = 3 
    base_dir = f'/pub/gps/data/daily/{yr}'
    data_dir = 'rnx'

    for dd in range(1,366):
        start_time = time.time()
        dd_str = f'{dd:03d}'
        new_dir = os.path.join(data_dir, f'gps_rnx_daily_{yr}{dd_str}')
        os.makedirs(new_dir, exist_ok=True)

        for _ in range(retries): 
            try:    
                ftp.cwd(base_dir)
                dd_dir = os.path.join(base_dir, dd_str)
                ftp.cwd(dd_dir)
    
                yr_suff = yr[-2:]
                year_suffix = f'{yr_suff}n'
                ftp.cwd(year_suffix)
                files = []
                                
                ftp.retrlines('NLST', files.append)
                            
                # rnx_files = [file for file in files if file.endswith('GN.rnx.gz') or file.endswith('n.Z')] # case-sensitive
                rnx_files = [file for file in files if file.endswith('GN.rnx.gz')] # case-sensitive
                                            
                for i, rnx_file in enumerate(rnx_files):
                    local_filename = os.path.join(new_dir, rnx_file)
                                                
                    with open(local_filename, 'wb') as f:
                        ftp.retrbinary(f"RETR {rnx_file}", f.write)
                                            
                    if rnx_file.endswith('.gz'):
                        local_uncompressed = local_filename[:-3]
                        try: 
                            with gzip.open(local_filename, 'rb') as gzipped_file, open(local_uncompressed, 'wb') as decompressed_file:
                                decompressed_file.write(gzipped_file.read())
                                files.append(local_uncompressed)
                        except Exception as e: 
                            print(f"Error uncompressing file {local_filename}: {e}. Will begin uncompressing file as a UNIX compressed file")
                            with open(local_filename, 'rb') as compressed_file, open(local_uncompressed, 'wb') as decompressed_file:
                                decompressed_file.write(unlzw(compressed_file.read()))
                                files.append(local_uncompressed)
                                print(f"Successfully uncompressed and saved file {local_filename} as a UNIX compressed file")
                                    
                        print(f"Done saving file {i + 1} out of {len(rnx_files)} for Day/Year: {dd}/{yr}")
    
                #         # elif rnx_file.endswith('.Z'):
                #         #     local_uncompressed = local_filename[:-2]
                #         #     try: 
                #         #         with open(local_filename, 'rb') as compressed_file, open(local_uncompressed, 'wb') as decompressed_file:
                #         #             decompressed_file.write(unlzw(compressed_file.read()))
                #         #             files.append(local_uncompressed)
                #         #     except Exception as e: 
                #         #         print(f"Error uncompressing a UNIX compressed file {local_filename}: {e}")
    
                #             # print(f"Done saving file {i + 1} out of {len(rnx_files)} for Day/Year: {dd}/{yr}")
                    
                end_time = time.time()
                elapsed_time = end_time - start_time
                print(f"Done uncompressing and saving files for Day/Year: {dd}/{yr} in folder: {new_dir}. The elapsed time of this operation was {elapsed_time:.2f} seconds")      
                break
            except Exception as e:
                print(f"Error on attempt {_ + 1} for Day/Year: {dd}/{yr}: {e}")
                if _ == retries - 1:
                    print(f"Failed to fetch data for Day/Year: {dd}/{yr} after {retries} attempts.")
                time.sleep(5)

    print(f"Done uncompressing and saving all files for Year: {yr} in folder: {new_dir}")

In [ ]:
%%time
yrs = range(2018,2019)

for yr in yrs: 
    fetch_data(f'{yr}')

# Archive

In [ ]:
# def fetch_data(yr):
#         try:
#             base_dir = f'/pub/gps/data/hourly/{yr}'
#             ftp.cwd(base_dir)
            
#             data_dir = 'rnx'
#             # new_dir = os.path.join(data_dir, f'gps_rnx_{yr}{dd}')
#             # os.makedirs(new_dir, exist_ok=True)

#             for dd in range(1,365):

#                 dd_str = f'{dd:03d}'
#                 new_dir = os.path.join(data_dir, f'gps_rnx_{yr}{dd_str}')
#                 os.makedirs(new_dir, exist_ok=True)
#                 dd_dir = os.path.join(base_dir, dd_str)
#                 ftp.cwd(dd_dir)

#                 for hr in range(24):      
#                     files = []
#                     hr_str = f'{hr:02d}'
#                     hour_dir = os.path.join(dd_dir, hr_str)
#                     ftp.cwd(hour_dir)
                            
#                     ftp.retrlines('NLST', files.append)
                        
#                     rnx_files = [file for file in files if file.endswith('GN.rnx.gz')] # case-sensitive
                                        
#                     for i, rnx_file in enumerate(rnx_files):
#                         local_filename = os.path.join(new_dir, rnx_file)
                                            
#                         with open(local_filename, 'wb') as f:
#                             ftp.retrbinary(f"RETR {rnx_file}", f.write)
                                        
#                         if rnx_file.endswith('.gz'):
#                             local_uncompressed = local_filename[:-3]
#                             try: 
#                                 with gzip.open(local_filename, 'rb') as gzipped_file, open(local_uncompressed, 'wb') as decompressed_file:
#                                     decompressed_file.write(gzipped_file.read())
#                                     files.append(local_uncompressed)
#                             except Exception as e: 
#                                 print(f"Error uncompressing file {local_filename}: {e}. Will begin uncompressing file as a UNIX compressed file")
#                                 with open(local_filename, 'rb') as compressed_file, open(local_uncompressed, 'wb') as decompressed_file:
#                                     decompressed_file.write(unlzw(compressed_file.read()))
#                                     files.append(local_uncompressed)
#                                     print(f"Successfully uncompressed and saved file {local_filename} as a UNIX compressed file")
                            
#                         print(f"Done saving file {i + 1} out of {len(rnx_files)} for Hour: {hr} Day/Year: {dd}/{yr}")
#                 print(f"Done uncompressing and saving files for Day/Year: {dd}/{yr} in folder: {new_dir}")      
#             print(f"Done uncompressing and saving all files for Year: {yr} in folder: {new_dir}")                

#         except Exception as e:
#             print(f"Error fetching data: {e}")